In [65]:
"""
Alunos: Luan L. Guarnieri, Maria E. Krutzsch
"""
import cv2
import os

# Função para carregar imagens de uma pasta
def carregar_imagens_da_pasta(pasta):
    imagens = {}
    for nome_arquivo in os.listdir(pasta):
        # Carrega a imagem em escala de cinza
        img = cv2.imread(os.path.join(pasta, nome_arquivo), cv2.IMREAD_GRAYSCALE)
        # Verifica se a imagem foi carregada corretamente
        imagens[nome_arquivo] = img
    return imagens

# Função para pré-processar uma imagem
def preprocessamento(img):
    # Aplica limiarização para binarizar a imagem
    _, img_limiarizada = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    return img_limiarizada

# Função para detectar keypoints e computar descritores usando SIFT
def extrair_sift(img):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(img, None)
    return kp, des

# Função para detectar keypoints e computar descritores usando ORB
def extrair_orb(img):
    orb = cv2.ORB_create()
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

# Função para realizar a correspondência de features entre duas imagens usando BFMatcher
def corresponder_features(des1, des2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    return matches

# Função para filtrar as melhores correspondências com base em um limiar de distância
def filtrar_correspondencias(matches, limiar=0.75):
    correspondencias_boas = [match for match in matches if match.distance < limiar]
    return correspondencias_boas

# Função para encontrar a melhor correspondência em um conjunto de correspondências
def encontrar_melhor_correspondencia(correspondencias):
    melhor_correspondencia = ('Incerteza', 0)
    for nome, lista_correspondencias in correspondencias:
        if len(lista_correspondencias) > melhor_correspondencia[1]:
            melhor_correspondencia = (nome, len(lista_correspondencias))
    return melhor_correspondencia

# Função para realizar o matching entre as imagens de treinamento e teste usando SIFT e ORB
def fazer_matching(imagens_treinamento, imagens_teste):
    sift = cv2.SIFT_create()
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher()

    resultados = []

    # Itera sobre as imagens de teste
    for nome_teste, img_teste in imagens_teste.items():
        # Pré-processa a imagem de teste
        img_teste = preprocessamento(img_teste)

        # Extrai keypoints e descritores usando SIFT e ORB
        kp_teste_sift, des_teste_sift = extrair_sift(img_teste)
        kp_teste_orb, des_teste_orb = extrair_orb(img_teste)

        # Verifica se os descritores foram computados corretamente
        if des_teste_sift is None or des_teste_orb is None:
            continue

        # Lista para armazenar as correspondências para cada imagem de treinamento
        correspondencias = []

        # Itera sobre as imagens de treinamento
        for nome_treinamento, img_treinamento in imagens_treinamento.items():
            # Pré-processa a imagem de treinamento
            img_treinamento = preprocessamento(img_treinamento)

            # Extrai keypoints e descritores usando SIFT e ORB
            kp_treinamento_sift, des_treinamento_sift = extrair_sift(img_treinamento)
            kp_treinamento_orb, des_treinamento_orb = extrair_orb(img_treinamento)

            # Encontra correspondências usando o objeto BFMatcher
            matches_sift = bf.knnMatch(des_treinamento_sift, des_teste_sift, k=2)
            matches_orb = bf.knnMatch(des_treinamento_orb, des_teste_orb, k=2)

            # Filtra correspondências boas usando o critério de distância
            correspondencias_boas_sift = [m for m, n in matches_sift if m.distance < 0.75 * n.distance]
            correspondencias_boas_orb = [m for m, n in matches_orb if m.distance < 0.75 * n.distance]

            # Armazena as correspondências para esta imagem de treinamento
            correspondencias.append((nome_treinamento, correspondencias_boas_sift, correspondencias_boas_orb))

        # Encontra a melhor correspondência para SIFT e ORB
        melhor_correspondencia_sift = encontrar_melhor_correspondencia([(nome, correspondencias_boas_sift) for nome, correspondencias_boas_sift, _ in correspondencias])
        melhor_correspondencia_orb = encontrar_melhor_correspondencia([(nome, correspondencias_boas_orb) for nome, _, correspondencias_boas_orb in correspondencias])

        # Verifica se alguma correspondência encontrada é suficientemente boa
        if melhor_correspondencia_sift[1] > 10 or melhor_correspondencia_orb[1] > 10:
            # Seleciona a correspondência com mais keypoints correspondentes
            if melhor_correspondencia_sift[1] > melhor_correspondencia_orb[1]:
                resultados.append((nome_teste, melhor_correspondencia_sift[0]))
            else:
                resultados.append((nome_teste, melhor_correspondencia_orb[0]))
        else:
            # Caso não haja correspondências suficientemente boas, adicione à lista de incertezas
            resultados.append((nome_teste, f"Incerteza (provavel: {melhor_correspondencia_sift[0]})"))

    return resultados

# Caminhos para os diretórios de treinamento e teste
# pasta entrada -> deverá conter as imagens para o treinamento
# pasta dataset -> deverá conter as imagens para compararção e match
pasta_treinamento = 'entrada'
pasta_teste = 'dataset'

# Carrega as imagens de treinamento e teste
imagens_treinamento = carregar_imagens_da_pasta(pasta_treinamento)
imagens_teste = carregar_imagens_da_pasta(pasta_teste)

# Realiza o matching entre as imagens de treinamento e teste
resultados = fazer_matching(imagens_treinamento, imagens_teste)

# Imprime os resultados ordenados pelo nome do arquivo de teste
for resultado in sorted(resultados, key=lambda x: x[0]):
    print(resultado[0], "->", resultado[1])



imagem50km_01.jpg -> Incerteza (provavel: 50km.jpg)
imagem50km_02.jpg -> 50km.jpg
imagem50km_03.jpg -> 50km.jpg
imagem50km_04.jpg -> 50km.jpg
imagem50km_05.jpg -> Incerteza (provavel: 50km.jpg)
imagem50km_06.jpg -> pare.jpg
imagem50km_07.jpg -> lombada.jpg
imagem50km_08.jpg -> Incerteza (provavel: pare.jpg)
imagem50km_09.jpg -> Incerteza (provavel: 50km.jpg)
imagem50km_10.jpg -> Incerteza (provavel: pare.jpg)
imagem50km_11.jpg -> pare.jpg
imagemLombada_01.jpg -> Incerteza (provavel: pare.jpg)
imagemLombada_02.jpg -> Incerteza (provavel: lombada.jpg)
imagemLombada_03.jpg -> Incerteza (provavel: pare.jpg)
imagemLombada_04.jpg -> Incerteza (provavel: 50km.jpg)
imagemLombada_05.jpg -> lombada.jpg
imagemLombada_06.jpg -> lombada.jpg
imagemLombada_07.jpg -> Incerteza (provavel: 50km.jpg)
imagemLombada_08.jpg -> Incerteza (provavel: 50km.jpg)
imagemLombada_09.jpg -> Incerteza (provavel: pare.jpg)
imagemLombada_10.jpg -> Incerteza (provavel: pare.jpg)
imagemLombada_11.jpg -> Incerteza (provave